# 0.0. Loading Data

In [ ]:
data_types_dict = {'timestamp': 'int64',
                          'user_id': 'int32',
                          'content_id': 'int16',
                          'content_type_id': 'int8',
                          'answered_correctly':'int8',
                          'prior_question_elapsed_time': 'float32',
                          'prior_question_had_explanation': 'boolean'
                  }

In [ ]:
import pandas as pd

treino = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                   usecols=[1, 2, 3, 4, 7, 8, 9],
                   dtype=data_types_dict
                   )

In [ ]:
questions_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv',
                            usecols=[0, 3],
                            dtype={'question_id': 'int16',
                              'part': 'int8'}
                          )

# 1.0. Data dimensions

In [ ]:
print(f'Number of Rows: {treino.shape[0]}')
print(f'Number of Cols: {treino.shape[1]}')

# 1.1. Data Type

In [ ]:
treino = treino.astype(data_types_dict)

In [ ]:
treino.dtypes

# 1.2. Data Missing

In [ ]:
treino.isna().sum()

# 2.0. First Feature Engineering

In [ ]:
treino = treino.loc[treino.content_type_id == False]
treino['tempo_exercicio'] = pd.DataFrame(treino.prior_question_elapsed_time.shift(-1))
treino.loc[treino.timestamp == 0, ['prior_question_had_explanation']] = treino.loc[treino.timestamp == 0, ['prior_question_had_explanation']].fillna(False)
treino = treino.sort_values(['timestamp'], ascending=True).reset_index(drop = True)


In [ ]:
treino = pd.merge(treino, questions_df, left_on = 'content_id', right_on = 'question_id', how = 'left')
treino.part = treino.part - 1

In [ ]:
treino.drop(['timestamp', 'content_type_id','question_id'], axis=1, inplace=True)

In [ ]:
elapsed_meanf = treino.prior_question_elapsed_time.mean()

In [ ]:
treino = treino[~treino['prior_question_had_explanation'].isna()]

# 3.0. Validation Data

In [ ]:
validation = pd.DataFrame()

In [ ]:
for i in range(4):
    last_records = treino.drop_duplicates('user_id', keep = 'last')
    treino = treino[~treino.index.isin(last_records.index)]
    validation = validation.append(last_records)
del(last_records)

In [ ]:
validation.drop(['tempo_exercicio'], axis=1, inplace=True)

In [ ]:
elapsed_mean = treino.prior_question_elapsed_time.mean()

# 4.0. Train Data

In [ ]:
X = pd.DataFrame()
for i in range(15):
    last_records = treino.drop_duplicates('user_id', keep = 'last')
    treino = treino[~treino.index.isin(last_records.index)]
    X = X.append(last_records)
del(last_records)

# 5.0. Second Feature Engineering

In [ ]:
tempo_medio_estudante = treino[['user_id','tempo_exercicio']].groupby(['user_id']).agg(['mean'])
tempo_medio_estudante.columns = ['tempo_medio_estudante']

tempo_medio_exercicio = treino[['content_id','tempo_exercicio']].groupby(['content_id']).agg(['mean'])
tempo_medio_exercicio.columns = ['tempo_medio_exercicio']

tempo_medio_tipo_exercicio = treino[['tempo_exercicio','part']].groupby(['part']).agg(['mean'])
tempo_medio_tipo_exercicio.columns = ['tempo_medio_tipo_exercicio']

acerto_medio_estudante = treino[['user_id','answered_correctly']].groupby(['user_id']).agg(['mean', 'count'])
acerto_medio_estudante.columns = ['acerto_medio_estudante', 'count']

media_numero_exercicio = acerto_medio_estudante[['acerto_medio_estudante','count']].groupby(['count']).agg(['mean'])
media_numero_exercicio.columns = ["media_numero_exercicio"]

acerto_medio_exercicio = treino[['content_id','answered_correctly']].groupby(['content_id']).agg(['mean'])
acerto_medio_exercicio.columns = ["acerto_medio_exercicio"]

acerto_medio_tipo_exercicio = treino[['part','answered_correctly']].groupby(['part']).agg(['mean'])
acerto_medio_tipo_exercicio.columns = ["acerto_medio_tipo_exercicio"]

viu_medio_estudante = treino[['prior_question_had_explanation']].astype('int8').groupby(treino['user_id']).agg(['mean','count'])
viu_medio_estudante.columns = ['viu_medio_estudante','count_had']

media_numero_viu = viu_medio_estudante[['viu_medio_estudante','count_had']].groupby(['count_had']).agg(['mean'])
media_numero_viu.columns = ["media_numero_viu"]

viu_medio_exercicio = treino[['prior_question_had_explanation']].astype('int8').groupby(treino['content_id']).agg(['mean'])
viu_medio_exercicio.columns = ["viu_medio_exercicio"]

del(treino)

In [ ]:
X = pd.merge(X, acerto_medio_estudante, on=['user_id'], how="left")
X = pd.merge(X, tempo_medio_estudante, on=['user_id'], how="left")
X = pd.merge(X, acerto_medio_exercicio, on=['content_id'], how="left")
X = pd.merge(X, tempo_medio_exercicio, on=['content_id'], how="left")
X = pd.merge(X, acerto_medio_tipo_exercicio, on=['part'], how="left")
X = pd.merge(X, tempo_medio_tipo_exercicio, on=['part'], how="left")
X = pd.merge(X, viu_medio_estudante, on=['user_id'], how="left")
X = pd.merge(X, viu_medio_exercicio, on=['content_id'], how="left")
X = pd.merge(X, media_numero_exercicio, on=['count'], how="left")
X = pd.merge(X, media_numero_viu, on=['count_had'], how="left")
X

In [ ]:
validation = pd.merge(validation, acerto_medio_estudante, on=['user_id'], how="left")
validation = pd.merge(validation, tempo_medio_estudante, on=['user_id'], how="left")
validation = pd.merge(validation, acerto_medio_exercicio, on=['content_id'], how="left")
validation = pd.merge(validation, tempo_medio_exercicio, on=['content_id'], how="left")
validation = pd.merge(validation, acerto_medio_tipo_exercicio, on=['part'], how="left")
validation = pd.merge(validation, tempo_medio_tipo_exercicio, on=['part'], how="left")
validation = pd.merge(validation, viu_medio_estudante, on=['user_id'], how="left")
validation = pd.merge(validation, viu_medio_exercicio, on=['content_id'], how="left")
validation = pd.merge(validation, media_numero_exercicio, on=['count'], how="left")
validation = pd.merge(validation, media_numero_viu, on=['count_had'], how="left")
validation

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()

X.prior_question_had_explanation.fillna(False, inplace = True)
validation.prior_question_had_explanation.fillna(False, inplace = True)

validation["prior_question_had_explanation_enc"] = lb_make.fit_transform(validation["prior_question_had_explanation"])
X["prior_question_had_explanation_enc"] = lb_make.fit_transform(X["prior_question_had_explanation"])

In [ ]:
y = X['answered_correctly']
X = X.drop(['answered_correctly'], axis=1)

y_val = validation['answered_correctly']
X_val = validation.drop(['answered_correctly'], axis=1)

In [ ]:
X

In [ ]:
a = ['acerto_medio_estudante','tempo_medio_estudante', 'acerto_medio_exercicio',
       'tempo_medio_exercicio','count','prior_question_elapsed_time',
       'prior_question_had_explanation_enc','part','count_had','viu_medio_estudante',
       'viu_medio_exercicio','media_numero_exercicio','media_numero_viu',
      'tempo_medio_tipo_exercicio','acerto_medio_tipo_exercicio']

P = X[a]

P_val = X_val[a]

yp = y

yp_val = y_val

In [ ]:
P['acerto_medio_estudante'].fillna(acerto_medio_estudante.acerto_medio_estudante.mean(),inplace=True)
P['acerto_medio_exercicio'].fillna(acerto_medio_exercicio.acerto_medio_exercicio.mean(),inplace=True)
P['acerto_medio_tipo_exercicio'].fillna(acerto_medio_tipo_exercicio.acerto_medio_tipo_exercicio.mean(),  inplace=True)
P['tempo_medio_estudante'].fillna(tempo_medio_estudante.tempo_medio_estudante.mean(),inplace = True)
P['tempo_medio_exercicio'].fillna(tempo_medio_exercicio.tempo_medio_exercicio.mean(),inplace = True)
P['tempo_medio_tipo_exercicio'].fillna(tempo_medio_tipo_exercicio.tempo_medio_tipo_exercicio.mean(),inplace=True)
P['media_numero_exercicio'].fillna(media_numero_exercicio.media_numero_exercicio.mean(),inplace=True)
P['viu_medio_estudante'].fillna(viu_medio_estudante.viu_medio_estudante.mean(),inplace=True)
P['media_numero_viu'].fillna(media_numero_viu.media_numero_viu.mean(),inplace=True)
P['viu_medio_exercicio'].fillna(viu_medio_exercicio.viu_medio_exercicio.mean(),inplace=True)


P_val['acerto_medio_estudante'].fillna(acerto_medio_estudante.acerto_medio_estudante.mean(),inplace=True)
P_val['acerto_medio_exercicio'].fillna(acerto_medio_exercicio.acerto_medio_exercicio.mean(),inplace=True)
P_val['acerto_medio_tipo_exercicio'].fillna(acerto_medio_tipo_exercicio.acerto_medio_tipo_exercicio.mean(),  inplace=True)
P_val['tempo_medio_estudante'].fillna(tempo_medio_estudante.tempo_medio_estudante.mean(),inplace = True)
P_val['tempo_medio_exercicio'].fillna(tempo_medio_exercicio.tempo_medio_exercicio.mean(),inplace = True)
P_val['tempo_medio_tipo_exercicio'].fillna(tempo_medio_tipo_exercicio.tempo_medio_tipo_exercicio.mean(),inplace=True)
P_val['media_numero_exercicio'].fillna(media_numero_exercicio.media_numero_exercicio.mean(),inplace=True)
P_val['viu_medio_estudante'].fillna(viu_medio_estudante.viu_medio_estudante.mean(),inplace=True)
P_val['media_numero_viu'].fillna(media_numero_viu.media_numero_viu.mean(),inplace=True)
P_val['viu_medio_exercicio'].fillna(viu_medio_exercicio.viu_medio_exercicio.mean(),inplace=True)

P['part'].fillna(4, inplace = True)
P['count'].fillna(0, inplace = True)
P['count_had'].fillna(0, inplace = True)
P['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
P['prior_question_had_explanation_enc'].fillna(0, inplace = True)

P_val['part'].fillna(4, inplace = True)
P_val['count'].fillna(0, inplace = True)
P_val['count_had'].fillna(0, inplace = True)
P_val['prior_question_elapsed_time'].fillna(elapsed_mean, inplace = True)
P_val['prior_question_had_explanation_enc'].fillna(0, inplace = True)

# 6.0. model creation

In [ ]:
import lightgbm as lgb

params = {
'num_leaves': 350,
'max_bin':700,
'min_child_weight': 0.03454472573214212,
'feature_fraction': 0.58,
'bagging_fraction': 0.58,
'objective': 'binary',
'max_depth': -1,
'learning_rate': 0.05,
"boosting_type": "gbdt",
"bagging_seed": 11,
"metric": 'auc',
"verbosity": -1,
'reg_alpha': 0.3899927210061127,
'reg_lambda': 0.6485237330340494,
'random_state': 47
}

lgb_train = lgb.Dataset(P, yp)
lgb_eval = lgb.Dataset(P_val, yp_val, reference=lgb_train)

In [ ]:
model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=50,
    num_boost_round=10000,
    early_stopping_rounds=12
)

In [ ]:
import numpy as np
y_pred = model.predict(P_val)
y_true = np.array(yp_val)

y_predc = model.predict(P)
y_truec = np.array(yp)

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
print(roc_auc_score(y_true, y_pred))
print(classification_report(y_true, y_pred.round()))

print(roc_auc_score(y_truec, y_predc))
print(classification_report(y_truec, y_predc.round()))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
lgb.plot_importance(model)
plt.show()